references: 
- http://cs229.stanford.edu/proj2013/ShiraniMehr-SMSSpamDetectionUsingMachineLearningApproach.pdf
- https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

In [1]:
import pandas as pd
import collections
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./dataset.csv')
print(df.shape)
df.head()

(4611, 2)


,Teks,label
0,Gimana dek...dah baikan blm...,0.0
1,Ikuti Seminar Inspiratif Cara Mudah Sukses Bis...,1.0
2,"Anda Terpilih\nSbgi pemenang\nCEK. Rp.100 ,jt\...",1.0
3,Punya masalah keuangan?\ncukup jaminkan Bpkb M...,1.0
4,DISK0N T0GEL hingga 65% hanya di J0KERBET888 b...,1.0


## Data Cleaning

In [3]:
df = df.dropna()
print(df.shape)

(4605, 2)


In [4]:
df['label'] = df['label'].astype('int64')
df.dtypes

Teks     object
label     int64
dtype: object

## Feature Extraction
### Extract length of sms, number of term with capital, and number of term with digits

In [5]:
len_values = []

for i in range(0, len(df)):
    len_values.append(len(df.iloc[i].Teks))
    
len_col = pd.Series(len_values)
df['length'] = len_col.values

In [6]:
df_iklan = df[df['label'] == 1]
df_non_iklan = df[df['label'] == 0]

print("Iklan shape", df_iklan.shape)
print("Non Iklan shape", df_non_iklan.shape)

Iklan shape (2459, 3)
Non Iklan shape (2146, 3)


In [7]:
cap_list = []

for i in range(0,len(df)):
    words = df.Teks.iloc[i].split()
    count = 0
    for j in range(0,len(words)):
        if(words[j].isupper()):
            count = count + 1;
    cap_list.append(count)    
    #print(cap_list)
        
cap_col = pd.Series(cap_list)
df['CAP'] = cap_col.values

In [8]:
digits_list = []

for i in range(0,len(df)):
    if(sum(c.isdigit() for c in df.Teks.iloc[i]) == 0):
        digits_list.append(0)
    else:
        digits_list.append(sum(c.isdigit() for c in df.Teks.iloc[i]))

digits_col = pd.Series(digits_list)
df['DIGITS'] = digits_col.values

In [9]:
df.head()

,Teks,label,length,CAP,DIGITS
0,Gimana dek...dah baikan blm...,0,30,0,0
1,Ikuti Seminar Inspiratif Cara Mudah Sukses Bis...,1,128,5,17
2,"Anda Terpilih\nSbgi pemenang\nCEK. Rp.100 ,jt\...",1,142,4,12
3,Punya masalah keuangan?\ncukup jaminkan Bpkb M...,1,149,1,25
4,DISK0N T0GEL hingga 65% hanya di J0KERBET888 b...,1,159,3,10


### [Optional] Convert numerical columns to categorical 

In [66]:
print("Number of unique length instances: ", len(set(df.length)))
print("Number of unique CAP instances: ", len(set(df.CAP)))
print("Number of unique DIGITS instances: ", len(set(df.DIGITS)))

Number of unique length instances:  283
Number of unique CAP instances:  19
Number of unique DIGITS instances:  64


In [67]:
print("Length describe:")
print(df.length.describe())
print("="*30, "\n")
print("DIGITS describe:")
print(df.DIGITS.describe())

Length describe:
count    4605.000000
mean      122.127253
std        55.860330
min         2.000000
25%        81.000000
50%       137.000000
75%       156.000000
max       828.000000
Name: length, dtype: float64

DIGITS describe:
count    4605.000000
mean       10.303149
std        10.577450
min         0.000000
25%         2.000000
50%         8.000000
75%        15.000000
max       107.000000
Name: DIGITS, dtype: float64


In [68]:
import time
import datetime

## Converting numerical column (length) to categorical
num_cols = ['length', 'DIGITS']
bound = {
    'length': [81, 137, 156],
    'DIGITS': [2, 8, 15]
}

start = time.time()

for col in num_cols:
    for i in df.index:
        if df[col][i] < bound[col][0]:
            df[col][i] = 1
        elif df[col][i] < bound[col][1]:
            df[col][i] = 2
        elif df[col][i] < bound[col][2]:
            df[col][i] = 3
        else:
            df[col][i] = 4

spent = time.time() - start
print("Converting execution time: ", str(datetime.timedelta(seconds=spent)))

print("Number of unique length instances, after conversion: ", len(set(df.length)))
print("Number of unique DIGITS instances, after conversion: ", len(set(df.DIGITS)))

Converting execution time:  0:04:51.586653
Number of unique length instances, after conversion:  4
Number of unique DIGITS instances, after conversion:  4


### Extract terms
#### Preprocessing

In [10]:
PATH_TO_KEY_NORM_FILE = './key_norm.csv'

key_norm = pd.read_csv(PATH_TO_KEY_NORM_FILE).drop(['_id'], axis=1)
key_norm.head()

,singkat,hasil
0,abis,habis
1,accent,tekanan
2,accept,terima
3,accident,kecelakaan
4,achievement,prestasi


In [11]:
key_norm_dict = {key_norm['singkat'][i]:key_norm['hasil'][i] for i in range(len(key_norm))}
key_norm_dict

{'abis': 'habis',
 'accent': 'tekanan',
 'accept': 'terima',
 'accident': 'kecelakaan',
 'achievement': 'prestasi',
 'acra': 'acara',
 'acrany': 'acaranya',
 'acrnya': 'acaranya',
 'action': 'aksi',
 'active': 'aktif',
 'activity': 'aktivitas',
 'actually': 'sebenarnya',
 'actualy': 'sebenarnya',
 'ad': 'ada',
 'ade': 'ada',
 'adult': 'dewasa',
 'adventure': 'petualangan',
 'adventurer': 'petualang',
 'advice': 'nasehat',
 'after': 'setelah',
 'afternun': 'sore',
 'again': 'lagi',
 'agency': 'perwakilan',
 'agent': 'agen',
 'agk': 'agak',
 'agktn': 'angkatan',
 'agree': 'setuju',
 'agreement': 'persetujuan',
 'aing': 'saya',
 'aj': 'saja',
 'aja': 'saja',
 'ajah': 'saja',
 'aje': 'saja',
 'ajeh': 'saja',
 'ajk': 'ajak',
 'ak': 'saya',
 'akeh': 'banyak',
 'akhire': 'akhirnya',
 'aktifkn': 'aktifkan',
 'aku': 'saya',
 'alhamdlh': 'alhamdulillah',
 'alhamdulilah': 'alhamdulillah',
 'almost': 'hampir',
 'almt': 'alamat',
 'alone': 'sendiri',
 'alsn': 'alasan',
 'also': 'juga',
 'always': '

In [12]:
PATH_TO_STOP_WORD_LIST_FILE = './stopword_list_TALA.txt'

with open(PATH_TO_STOP_WORD_LIST_FILE, 'r') as stop_word_list_file:
    stop_words = [word.rstrip('\n').lower() for word in stop_word_list_file.readlines()]

stop_words

['at_user',
 'atuser',
 'url',
 'rt',
 'ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdata

In [13]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize

import re

# Create StopWord Removal using our own Stop words
stop_word_remover = StopWordRemover(ArrayDictionary(stop_words))

# Create stemmer
stemmer = StemmerFactory().create_stemmer()

def formalize(word):
    if word in key_norm_dict:
        return key_norm_dict[word]
    return word

def preprocess_sms(sms):    
    # Lower casing
    clean_sms = sms.lower()

    # Punctuation removal
    clean_sms = re.sub(r'[^\w\s]', ' ', clean_sms)
    
    # Extra space removal
    clean_sms = re.sub('\s+', ' ', clean_sms)
    
     # Trimming
    clean_sms = clean_sms.strip()
     
    # Stop words removal
    clean_sms = stop_word_remover.remove(clean_sms)
    
    # Transforming informal words to formal words
    clean_sms = " ".join([formalize(word) for word in word_tokenize(clean_sms)])
    
    # Removing repeated characters
    rpt_regex = re.compile(r"(.)\1{1,}", re.IGNORECASE) #regex for repeating characters
    clean_sms = re.sub(rpt_regex, '', clean_sms)
    
    # Stemming
    clean_sms = stemmer.stem(clean_sms)

    return clean_sms

In [14]:
import time
import datetime

start = time.time()

raw = df.Teks.copy()
cleaned = pd.Series([preprocess_sms(sms) for sms in df['Teks']])
df['Teks'] = cleaned

spent = time.time() - start
print("Preprocessing execution time: ", str(datetime.timedelta(seconds=spent)))
df.head()

Preprocessing execution time:  0:12:45.010331


,Teks,label,length,CAP,DIGITS
0,bagaimana dek sudah baik belum,0,30,0,0
1,ikut seminar inspiratif mudah sukses bisnis to...,1,128,5,17
2,pilih bagai menang cek rp 1 jt kuota flash int...,1,142,4,12
3,masalah uang jamin bpkb mobil pinjmana sesuai ...,1,149,1,25
4,disk0n t0gel 65 di j0kerbet bit ly 2sngroy 6 p...,1,159,3,10


#### Before and after preprocessing comparison

In [15]:
raw.head()

0                       Gimana dek...dah baikan blm...
1    Ikuti Seminar Inspiratif Cara Mudah Sukses Bis...
2    Anda Terpilih\nSbgi pemenang\nCEK. Rp.100 ,jt\...
3    Punya masalah keuangan?\ncukup jaminkan Bpkb M...
4    DISK0N T0GEL hingga 65% hanya di J0KERBET888 b...
Name: Teks, dtype: object

In [16]:
cleaned.head()

0                       bagaimana dek sudah baik belum
1    ikut seminar inspiratif mudah sukses bisnis to...
2    pilih bagai menang cek rp 1 jt kuota flash int...
3    masalah uang jamin bpkb mobil pinjmana sesuai ...
4    disk0n t0gel 65 di j0kerbet bit ly 2sngroy 6 p...
dtype: object

#### Extract bag of words

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

def extract_bag_of_words(sms_list):
    vectorizer = CountVectorizer(max_features=2000, min_df=0.08, max_df=0.75)  
    return vectorizer.fit_transform(np.array(sms_list)).toarray()

In [18]:
bag_of_words_feature = extract_bag_of_words(cleaned)
print(bag_of_words_feature.shape)

(4605, 21)


In [19]:
df_bow = pd.concat([df, pd.DataFrame(bag_of_words_feature)], axis=1)
df_bow.head()

,Teks,label,length,CAP,DIGITS,0,1,2,3,4,...,11,12,13,14,15,16,17,18,19,20
0,bagaimana dek sudah baik belum,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ikut seminar inspiratif mudah sukses bisnis to...,1.0,128.0,5.0,17.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,pilih bagai menang cek rp 1 jt kuota flash int...,1.0,142.0,4.0,12.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,masalah uang jamin bpkb mobil pinjmana sesuai ...,1.0,149.0,1.0,25.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,disk0n t0gel 65 di j0kerbet bit ly 2sngroy 6 p...,1.0,159.0,3.0,10.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
print("df_bow shape: ", df_bow.shape)
df_bow = df_bow.dropna()
print("df_bow shape after drop na: ", df_bow.shape)

df_bow shape:  (4611, 26)
df_bow shape after drop na:  (4599, 26)


## Training & Test
### Utility function

In [21]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split


# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0


def removearray(L,arr):
    ind = 0
    size = len(L)
    while ind != size and not np.array_equal(L[ind],arr):
        ind += 1
    if ind != size:
        L.pop(ind)
    else:
        raise ValueError('array not found in list.')


# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

### Split function

In [22]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right


# Select the best split point for a dataset
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}


# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)


# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

### Building Tree

In [23]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root


# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']


# Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

In [24]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        removearray(train_set,fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [25]:
X = df_bow.drop(['label', 'Teks'], axis=1).copy()
y = df_bow[['label']].copy()
X.head()

,length,CAP,DIGITS,0,1,2,3,4,5,6,...,11,12,13,14,15,16,17,18,19,20
0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128.0,5.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,142.0,4.0,12.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,149.0,1.0,25.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,159.0,3.0,10.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
dataset = np.hstack((X, y))
dataset.shape

(4599, 25)

In [27]:
dataset

array([[ 30.,   0.,   0., ...,   0.,   0.,   0.],
       [128.,   5.,  17., ...,   0.,   0.,   1.],
       [142.,   4.,  12., ...,   0.,   0.,   1.],
       ...,
       [ 22.,   0.,   4., ...,   0.,   0.,   0.],
       [ 15.,   0.,   4., ...,   0.,   0.,   0.],
       [134.,   9.,   4., ...,   0.,   0.,   0.]])

In [28]:
from random import seed
from random import randrange
import time
import datetime

seed(1)
start = time.time()

# evaluate algorithm
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)

end = time.time()
spent = end - start
print("Execution time: ", str(datetime.timedelta(seconds=spent)))
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Execution time:  0:40:27.711354
Scores: [84.33079434167573, 85.52774755168662, 85.85418933623504, 83.02502720348205, 85.96300326441785]
Mean Accuracy: 84.940%
